In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import shutil
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import exposure
from skimage.restoration import denoise_wavelet

In [ ]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import time

In [ ]:
# Path to the local zip file and extraction directory
local_zip = '/content/drive/MyDrive/RA/Knee Osteoarthritis Dataset with Severity Grading.zip'
extraction_path = '/content/Knee_Osteoarthritis_Dataset'
shutil.unpack_archive(local_zip, extraction_path, 'zip')


In [ ]:
!pip install split-folders

import splitfolders

In [ ]:
# Function to resize the image
def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size)

# Function to sharpen the image
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Function to denoise the image
def denoise_image(image):
    denoised_image = denoise_wavelet(image, multichannel=True)
    denoised_image = (denoised_image * 255).astype(np.uint8)
    return denoised_image

# Function to apply histogram equalization
def equalize_histogram(image):
    yuv_img = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    yuv_img[:, :, 0] = cv2.equalizeHist(yuv_img[:, :, 0])
    hist_equalized_img = cv2.cvtColor(yuv_img, cv2.COLOR_YUV2BGR)
    return hist_equalized_img

# Function to enhance the contrast of the image
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)


# Function to display images from each step
def display_images(images, titles):
    fig, axs = plt.subplots(1, len(images), figsize=(20, 5))
    for i, (img, title) in enumerate(zip(images, titles)):
        axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[i].set_title(title)
        axs[i].axis('off')# Sample image from each class after each preprocessing step
def process_and_display_samples(input_dir, class_labels):
    for label in class_labels:
        class_dir = os.path.join(input_dir, label)
        sample_image_path = os.path.join(class_dir, os.listdir(class_dir)[0])
        image = cv2.imread(sample_image_path)

        # Step 1: Original
        original = image

        # Step 2: Resize image
        resized_image = resize_image(image)

        # Step 3: Sharpen image
        sharpened_image = sharpen_image(resized_image)

        # Step 4: Denoise image
        denoised_image = denoise_image(sharpened_image)

        # Step 5: Apply histogram equalization
        hist_equalized_image = equalize_histogram(denoised_image)

        # Step 6: Enhance contrast
        contrast_image = enhance_contrast(hist_equalized_image)

        # Step 7: Apply CLAHE
        clahe_image = contrast_image


        # Display images after each step
        display_images(
            [original, resized_image, sharpened_image, denoised_image, hist_equalized_image,  contrast_image],
            ['Original', 'Resized', 'Sharpened', 'Denoised', 'Histogram Equalized', 'Contrast Enhanced']
        )
    plt.show()


In [ ]:
# Define paths
train_dir = '/content/Knee_Osteoarthritis_Dataset/Knee Osteoarthritis Dataset with Severity Grading/train'
val_dir = '/content/Knee_Osteoarthritis_Dataset/Knee Osteoarthritis Dataset with Severity Grading/val'
test_dir = '/content/Knee_Osteoarthritis_Dataset/Knee Osteoarthritis Dataset with Severity Grading/test'
class_labels = sorted(os.listdir(train_dir))

# Display one sample from each class after each preprocessing step
process_and_display_samples(train_dir, class_labels)


In [ ]:
# Preprocess and save images for train, validation, and test datasets
output_train_dir = '/content/Knee_Osteoarthritis_Dataset_Preprocessed/train'
output_val_dir = '/content/Knee_Osteoarthritis_Dataset_Preprocessed/validation'
output_test_dir = '/content/Knee_Osteoarthritis_Dataset_Preprocessed/test'

preprocess_and_save_images(train_dir, output_train_dir)
preprocess_and_save_images(val_dir, output_val_dir)
preprocess_and_save_images(test_dir, output_test_dir)

print("Preprocessing complete. Preprocessed images are saved in /content/Knee_Osteoarthritis_Dataset_Preprocessed")
